<div align="right">  
    Grupo 13  
    
    André Neves da Costa - A95869 
    Filipe José Silva Castro - A96156 
</div>

## Variáveis de input

+ **m** $\rightarrow$ Tamanho que vai ser atribuído aos BitVectors
+ **a** $\rightarrow$ Valor que vai ser atribuído ao x do estado inicial
+ **b** $\rightarrow$ Valor que vai ser atribuído ao y do estado inicial
+ **k** $\rightarrow$ Número de transações de estado que queremos verificar se é possível fazer

## Variáveis auxiliares

+ **state** $\rightarrow$ Dicionário que guarda o estado e o valor de x, y e z

## Predicados

O **estado inicial** é caracterizado pelo seguinte predicado:

$$ (pc = 0 \wedge x = a \wedge y = b \wedge z = 0) $$

As **transformações possíveis** são caracterizadas pelo seguinte predicado:

$$ (pc = 0 \wedge pc' = 1 \wedge x' = x \wedge y' = y \wedge z' = z) $$
$$ \vee $$
$$ (pc = 1 \wedge pc' = 2 \wedge x' = x \wedge y' = y \wedge y \ne 0 \wedge even(y) \wedge z' = z ) $$
$$ \vee $$
$$ (pc = 1 \wedge pc' = 3 \wedge x' = x \wedge y' = y \wedge y \ne 0 \wedge odd(y) \wedge z' = z ) $$
$$ \vee $$
$$ (pc = 1 \wedge pc' = 4 \wedge x' = x \wedge y' = y \wedge y = 0 \wedge z' = z) $$
$$ \vee $$
$$ (pc = 2 \wedge pc' = 5 \wedge x' = 2 \times x \wedge y' = y/2 \wedge z' = z \wedge overflow) $$
$$ \vee $$
$$ (pc = 2 \wedge pc' = 1 \wedge x' = 2 \times x \wedge y' = y/2 \wedge z' = z) $$
$$ \vee $$
$$ (pc = 3 \wedge pc' = 5 \wedge x' = x \wedge y' = y - 1 \wedge z' = z + x \wedge overflow) $$
$$ \vee $$
$$ (pc = 3 \wedge pc' = 1 \wedge x' = x \wedge y' = y - 1 \wedge z' = z + x) $$

## Implementação em Python

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import BVType
from pysmt.typing import INT

m = 10        
a = 4
b = 6 
k = 5

### `declare`
Cria um dicionário que terá um **inteiro** correspondente ao **pc** (número do estado), e **BitVectores** com **m** bits que correspondem aos valores de **x**, **y** e **z**.

In [2]:
def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i),INT)
    state['x'] = Symbol('x'+str(i),BVType(m))
    state['y'] = Symbol('y'+str(i),BVType(m))
    state['z'] = Symbol('z'+str(i),BVType(m))
    return state

### `init`
Verifica se o **state** que recebe é um possível estado inicial, usando para isso o *predicado do estado inicial*.

In [3]:
def init(state):
    return And(Equals(state['pc'],Int(0)),Equals(state['x'],BV(a,m)),
               Equals(state['y'],BV(b,m)),Equals(state['z'],BV(0,m)))

### `trans`
Verifica se existe uma transição possível entre as variáveis de estado **curr** e **prox**, usando para isso o *predicado das transformações possíveis*.

### Observações
Construir esta função provou-se ser a parte mais desafiadora do exercício, sendo *BitVectors* um DataType que ainda não tinhamos utilizado tivemos que pesquisar imenso e fazer várias tentativas para aprendermos finalmente a manipular *BitVectors* corretamente, a melhor maneira que encontramos para o fazer foi usar **&1** que devolve 0 se o ultimo bit do *BitVector* for 0 e 1 caso contrário. Depois de saber isso foi fácil fazer a implementação para se y for par ou ímpar e também para verificar se haveria ou não *overflow*.

In [4]:
def trans(curr,prox):
    t01 = And(Equals(curr['pc'],Int(0)), Equals(prox['pc'],Int(1)), 
              Equals(curr['x'],prox['x']),Equals(curr['y'],prox['y']),
              Equals(curr['z'],prox['z']))
    t12 = And(Equals(curr['pc'],Int(1)), Equals(prox['pc'],Int(2)),
              Equals(curr['x'],prox['x']),Equals(curr['y'],prox['y']),
              NotEquals(curr['y'],BV(0,m)), Equals(curr['y']&1,BV(0,m)&1),
              Equals(curr['z'],prox['z']))
    t13 = And(Equals(curr['pc'],Int(1)), Equals(prox['pc'],Int(3)), 
              Equals(curr['x'],prox['x']),Equals(curr['y'],prox['y']), 
              NotEquals(curr['y'],BV(0,m)), Equals(curr['y']&1,BV(1,m)&1),
              Equals(curr['z'],prox['z']))
    t14 = And(Equals(curr['pc'],Int(1)), Equals(prox['pc'],Int(4)), 
              Equals(curr['x'],prox['x']),Equals(curr['y'],prox['y']),
              Equals(curr['y'],BV(0,m)),Equals(curr['z'],prox['z']))
    t21 = And(Equals(curr['pc'],Int(2)), Equals(prox['pc'],Int(1)), 
              Equals(curr['x']>>m-1 &1,BV(0,m)&1), Equals(prox['x'],curr['x']<<1),
              Equals(prox['y'],curr['y']>>1),Equals(curr['z'],prox['z']))
    t25 = And(Equals(curr['pc'],Int(2)), Equals(prox['pc'],Int(5)), 
              Equals(curr['x']>>m-1 &1,BV(1,m)&1), Equals(prox['x'],curr['x']<<1),
              Equals(prox['y'],curr['y']>>1),Equals(curr['z'],prox['z']))
    t35 = And(Equals(curr['pc'],Int(3)), Equals(prox['pc'],Int(5)), 
              Equals(curr['x'],prox['x']),Equals(prox['y'],BVSub(curr['y'],BVOne(m))),
              GE(BVToNatural(curr['x']) + BVToNatural(curr['z']),Int(2^m)),
              Equals(prox['z'],BVAdd(curr['z'],curr['x'])))
    t31 = And(Equals(curr['pc'],Int(3)), Equals(prox['pc'],Int(1)), 
              Equals(curr['x'],prox['x']),Equals(prox['y'],BVSub(curr['y'],BVOne(m))),
              Equals(prox['z'],BVAdd(curr['z'],curr['x'])))

    return Or(t01,t12,t13,t14,t21,t25,t31,t35)

### `gera_traco`
Recebe as funções `declare`, `init` e `trans`.
Nesta função vamos trabalhar com um solver z3 chamado **s**.

Primeiramente é criada uma lista **trace** onde se vai armazenar todas as variáveis de estado de **0** a **k**,
é depois adicionado a **s** o `init` do primeiro elemento de **trace** e os `trans` de todos os **trace** entre **0** e **k-1**.

In [5]:
def gera_traco(declare,init,trans):

    with Solver(name="z3") as s:
        
        # criar k cópias do estado
        trace = [declare(i) for i in range(k)]
    
        # criar o traço
        s.add_assertion(init(trace[0]))
        for i in range(k-1):
            s.add_assertion(trans(trace[i],trace[i+1]))
        
        if s.solve():
            for i in range(k):
                print(i)
                print("pc=", s.get_value(trace[i]['pc']))
                print("x=", s.get_value(trace[i]['x']))
                print("y=", s.get_value(trace[i]['y']))
                print("z=", s.get_value(trace[i]['z']))
        else:
            print("Impossivel criar um caminho")
  
                
gera_traco(declare,init,trans)

0
pc= 0
x= 4_10
y= 6_10
z= 0_10
1
pc= 1
x= 4_10
y= 6_10
z= 0_10
2
pc= 2
x= 4_10
y= 6_10
z= 0_10
3
pc= 1
x= 8_10
y= 3_10
z= 0_10
4
pc= 3
x= 8_10
y= 3_10
z= 0_10


Para finalizar o exercício foi adicionado ao `gera_traco` a possibilidade de receber um invariante **inv**, que neste caso se chama **invariante** e vai testar o seguinte em todas as variáveis de estado:

$$ (x∗y+z=a∗b) $$

In [6]:
def gera_traco(declare,init,trans,inv):

    with Solver(name="z3") as s:
        
    
        # criar k cópias do estado
        trace = [declare(i) for i in range(k)]
    
        # criar o traço
        s.add_assertion(init(trace[0]))
        for i in range(k-1):
            s.add_assertion(trans(trace[i],trace[i+1]))
        
        for i in range(k):
            s.add_assertion(inv(trace[i]))
        
        if s.solve():
            for i in range(k):
                print(i)
                print("pc=", s.get_value(trace[i]['pc']))
                print("x=", s.get_value(trace[i]['x']))
                print("y=", s.get_value(trace[i]['y']))
                print("z=", s.get_value(trace[i]['z']))
        else:
            print("Impossivel criar um caminho")
  

def invariante(state):
    return Equals(BVAdd(BVMul(state['x'],state['y']), state['z']), BV(a*b,m))
                
gera_traco(declare,init,trans,invariante)

0
pc= 0
x= 4_10
y= 6_10
z= 0_10
1
pc= 1
x= 4_10
y= 6_10
z= 0_10
2
pc= 2
x= 4_10
y= 6_10
z= 0_10
3
pc= 1
x= 8_10
y= 3_10
z= 0_10
4
pc= 3
x= 8_10
y= 3_10
z= 0_10


# Testes e os seus Outputs

In [7]:
m = 10        
a = 4
b = 6 
k = 5

----------

0
pc= 0
x= 4_10
y= 6_10
z= 0_10
1
pc= 1
x= 4_10
y= 6_10
z= 0_10
2
pc= 2
x= 4_10
y= 6_10
z= 0_10
3
pc= 1
x= 8_10
y= 3_10
z= 0_10
4
pc= 3
x= 8_10
y= 3_10
z= 0_10

----------

m = 10        
a = 4
b = 5 
k = 5

----------

0
pc= 0
x= 4_10
y= 5_10
z= 0_10
1
pc= 1
x= 4_10
y= 5_10
z= 0_10
2
pc= 3
x= 4_10
y= 5_10
z= 0_10
3
pc= 1
x= 4_10
y= 4_10
z= 4_10
4
pc= 2
x= 4_10
y= 4_10
z= 4_10

---------

m = 5       
a = 16
b = 6 
k = 4

---------

0
pc= 0
x= 16_5
y= 6_5
z= 0_5
1
pc= 1
x= 16_5
y= 6_5
z= 0_5
2
pc= 2
x= 16_5
y= 6_5
z= 0_5
3
pc= 5
x= 0_5
y= 3_5
z= 0_5

---------

m = 10        
a = 4
b = 6 
k = 10

---------

0
pc= 0
x= 4_10
y= 6_10
z= 0_10
1
pc= 1
x= 4_10
y= 6_10
z= 0_10
2
pc= 2
x= 4_10
y= 6_10
z= 0_10
3
pc= 1
x= 8_10
y= 3_10
z= 0_10
4
pc= 3
x= 8_10
y= 3_10
z= 0_10
5
pc= 1
x= 8_10
y= 2_10
z= 8_10
6
pc= 2
x= 8_10
y= 2_10
z= 8_10
7
pc= 1
x= 16_10
y= 1_10
z= 8_10
8
pc= 3
x= 16_10
y= 1_10
z= 8_10
9
pc= 1
x= 16_10
y= 0_10
z= 24_10

-----------

m = 5        
a = 6
b = 0 
k = 3

-----------

0
pc= 0
x= 6_5
y= 0_5
z= 0_5
1
pc= 1
x= 6_5
y= 0_5
z= 0_5
2
pc= 4
x= 6_5
y= 0_5
z= 0_5

----------

m = 5        
a = 16
b = 6 
k = 5

----------

Impossivel criar um caminho

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (2200602611.py, line 12)